In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
organizations_yelp_dataset_yelp_dataset_path = kagglehub.dataset_download('organizations/yelp-dataset/yelp-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/yelp-dataset


In [ ]:
import os

dataset_path = "/kaggle/input/yelp-dataset"

# List all files in the dataset folder
for filename in os.listdir(dataset_path):
    print(filename)


Dataset_User_Agreement.pdf
yelp_academic_dataset_review.json
yelp_academic_dataset_checkin.json
yelp_academic_dataset_business.json
yelp_academic_dataset_tip.json
yelp_academic_dataset_user.json


In [ ]:
import json

file_path = "/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json"

# Read the first 5 lines manually
with open(file_path, "r", encoding="utf-8") as f:
    for _ in range(5):
        print(json.loads(f.readline()))  # Read line by line

{'review_id': 'KU_O5udG6zpxOg-VcAEodg', 'user_id': 'mh_-eMZ6K5RLWhZyISBhwA', 'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw', 'stars': 3.0, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", 'date': '2018-07-07 22:09:11'}
{'review_id': 'BiTunyQ73aT9WBnpR9DZGw', 'user_id': 'OyoGAe7OKpv6SyGZT5g77Q', 'business_id': '7ATYjTIgM3jUlt4UM3IypQ', 'stars': 5.0, 'useful': 1, 'funny': 0, 'cool': 1, 'text': "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycl

In [ ]:
import json
import pandas as pd

# Define the correct file path
file_path = "/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json"

# Read the dataset (JSON lines format)
reviews = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        reviews.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(reviews)

# Select the three most-reviewed businesses
top_businesses = df["business_id"].value_counts().nlargest(3).index.tolist()

# Filter reviews for these businesses
filtered_df = df[df["business_id"].isin(top_businesses)][["user_id", "business_id", "text"]]

# **Fix: Remove duplicate reviews (keep only the first review per user & business)**
filtered_df = filtered_df.drop_duplicates(subset=["user_id", "business_id"], keep="first")

# Pivot the data to create the required table
table_df = filtered_df.pivot(index="user_id", columns="business_id", values="text")

# Rename columns to "Review 1", "Review 2", "Review 3"
table_df.columns = [f"Review {i+1}" for i in range(len(table_df.columns))]

# Reset index to include `user_id` as a column
table_df.reset_index(inplace=True)

# Select at least 6002 unique users
table_df = table_df.head(6002)

# Save the table as a CSV file
output_path = "/kaggle/working/yelp_reviews_table.csv"
table_df.to_csv(output_path, index=False)

# Display a sample of the final table
print(table_df.head())

print(f"✅ Data saved successfully to {output_path}")


                  user_id                                           Review 1  \
0  --b_RUNZHK5I2MVQiFjztA                                                NaN   
1  --cO5bedOWodnt8UgamQNA  I had to try the Nashville Hot Chicken from th...   
2  --e8m6lVrPIKZwVpGsCC5A                                                NaN   
3  --nJFgLzMnczShVC_CGslQ  I used to always say my favorite food was sush...   
4  -03y31IzykunU9azzgLsoQ  Nashville loves their hot chicken and this pla...   

                                            Review 2  \
0  Family and I came here last night and waited f...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                            Review 3  
0                                                NaN  
1                                                NaN  
2  Came here whil

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Ensure the VADER lexicon is available
nltk.download("vader_lexicon")

# Load the dataset
file_path = "/kaggle/working/yelp_reviews_table.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to get sentiment score
def get_sentiment(text):
    if pd.isna(text) or text.strip() == "":
        return 0  # Replace missing reviews with 0
    return sia.polarity_scores(text)["compound"]

# Apply sentiment analysis, replacing review text with sentiment scores
for col in df.columns[1:]:  # Skipping user_id
    df[col] = df[col].apply(get_sentiment)

# Save the updated dataset
output_path = "/kaggle/working/yelp_reviews_with_sentiment.csv"
df.to_csv(output_path, index=False)

# Display sample results
print(df.head())

print(f"✅ Sentiment analysis completed. File saved at: {output_path}")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
                  user_id  Review 1  Review 2  Review 3
0  --b_RUNZHK5I2MVQiFjztA    0.0000    0.9734    0.0000
1  --cO5bedOWodnt8UgamQNA    0.8830    0.0000    0.0000
2  --e8m6lVrPIKZwVpGsCC5A    0.0000    0.0000   -0.4754
3  --nJFgLzMnczShVC_CGslQ    0.9313    0.0000    0.0000
4  -03y31IzykunU9azzgLsoQ    0.8942    0.0000    0.0000
✅ Sentiment analysis completed. File saved at: /kaggle/working/yelp_reviews_with_sentiment.csv
